In [6]:
import ase
import numpy as np
from numpy.linalg import norm, svd
from ase.visualize import view
import struct_statistics as stats
import matplotlib.pyplot as p
import soaputils as su
#%matplotlib

In [7]:
# generate and read the two structures used in this notebook
struct_ran = su.gen_struct(70, seed=50)
struct_opt = ase.io.read('res_structs/x0_test/batch0.cfg')

In [ ]:
# calculate SOAP-parameters and matrices, only works on linux systems, 
# so the matrices were pre-calculated can be loaded in the cell below
import soaplite
import genBasis
rCut = 10.0
NradBas = 5
Lmax = 5
myAlphas, myBetas = genBasis.getBasisFunc(rCut, NradBas)

at_ran = soaplite.get_periodic_soap_structure(struct_ran, myAlphas, myBetas, rCut, NradBas, Lmax)
mat_opt = soaplite.get_periodic_soap_structure(struct_opt, myAlphas, myBetas, rCut, NradBas, Lmax)
np.save("res_data/mat_ran",mat_ran)
np.save("res_data/mat_opt",mat_opt)

In [8]:
# load matrices from storage
mat_ran = np.load("res_data/mat_ran.npy")
mat_opt = np.load("res_data/mat_opt.npy")

In [9]:
# show random structure
view(struct_ran)

In [10]:
# show optimised structure
view(struct_opt)

In [12]:
# show matrices
%matplotlib qt5
fig, axs = p.subplots(1, 2)
p.subplot(1,2,1)
p.imshow(mat_ran)

p.subplot(1,2,2)
p.imshow(mat_opt)

#p.colorbar()
p.colorbar(orientation='horizontal')
p.show()

In [13]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = p.subplots(nrows=1, ncols=2)
for ax in axes.flat:
    im = ax.imshow(mat_ran)

fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

p.colorbar(im, cax=cax)
p.show()

In [14]:
# show SVD plots
s_ran = svd(mat_ran.transpose(), full_matrices=False, compute_uv=False)
s_opt = svd(mat_opt.transpose(), full_matrices=False, compute_uv=False)

p.semilogy(s_ran)
p.semilogy(s_opt)
p.xlabel('$i$')
p.ylabel('Singular value $\sigma_i$')
p.legend(['random structure','optimised structure'])

